---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

In [37]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 
          'KY': 'Kentucky', 'AS': 'American Samoa', 
          'NV': 'Nevada', 'WY': 'Wyoming', 
          'NA': 'National', 'AL': 'Alabama', 
          'MD': 'Maryland', 'AK': 'Alaska', 
          'UT': 'Utah', 'OR': 'Oregon', 
          'MT': 'Montana', 'IL': 'Illinois', 
          'TN': 'Tennessee', 'DC': 'District of Columbia', 
          'VT': 'Vermont', 'ID': 'Idaho', 
          'AR': 'Arkansas', 'ME': 'Maine', 
          'WA': 'Washington', 'HI': 'Hawaii', 
          'WI': 'Wisconsin', 'MI': 'Michigan', 
          'IN': 'Indiana', 'NJ': 'New Jersey', 
          'AZ': 'Arizona', 'GU': 'Guam', 
          'MS': 'Mississippi', 'PR': 'Puerto Rico', 
          'NC': 'North Carolina', 'TX': 'Texas', 
          'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 
          'IA': 'Iowa', 'MO': 'Missouri', 
          'CT': 'Connecticut', 'WV': 'West Virginia', 
          'SC': 'South Carolina', 'LA': 'Louisiana', 
          'KS': 'Kansas', 'NY': 'New York', 
          'NE': 'Nebraska', 'OK': 'Oklahoma', 
          'FL': 'Florida', 'CA': 'California', 
          'CO': 'Colorado', 'PA': 'Pennsylvania', 
          'DE': 'Delaware', 'NM': 'New Mexico', 
          'RI': 'Rhode Island', 'MN': 'Minnesota', 
          'VI': 'Virgin Islands', 'NH': 'New Hampshire', 
          'MA': 'Massachusetts', 'GA': 'Georgia', 
          'ND': 'North Dakota', 'VA': 'Virginia'}

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    df = pd.read_table('university_towns.txt')
    top = pd.DataFrame({'Alabama[edit]': 'Alabama[edit]'}, index = [0])
    df = pd.concat([top,df])
    list_is = []
    state = df.iloc[0][0][:-6]
    
    for i in range(len(df)):
        if '[edit]' not in df.iloc[i][0]:
            list_is.append([state, df.iloc[i][0]])
        else:
            state = df.iloc[i][0][:-6]
            
    for j in range(len(list_is)):
        if '(' in list_is[j][1]:  
            for i in range(len(list_is[j][1])):
                if list_is[j][1][i] == '(':
                    break
            list_is[j][1] = list_is[j][1][:i].rstrip()
    
    result = pd.DataFrame(list_is, columns=['State', 'RegionName'])
    return result

get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [8]:
def cleaned_data():
    '''
    Returns a dataframe after cleaning gdplev.xls.
    '''
    df = pd.read_excel('gdplev.xls')
    rows_to_skip = df[df['Unnamed: 4'] == '2000q1'].index.values[0] #Getting the index value from where the 2000 quarters start.
    df = df.drop(df.index[0:rows_to_skip]) #Dropping the unwanted rows
    df.head()
    df = df[[4,5,6]] #Keeping the wanted columns
    df.rename(columns= {'Unnamed: 4': 'Quarters',
                        'Unnamed: 5': 'GDP',
                        'Unnamed: 6': 'Chained GDP'}, inplace = True)
    df = df.set_index('Quarters')
    return df

cleaned_data()

,GDP,Chained GDP
Quarters,,
2000q1,10031,12359.1
2000q2,10278.3,12592.5
2000q3,10357.4,12607.7
2000q4,10472.3,12679.3
2001q1,10508.1,12643.3
2001q2,10638.4,12710.3
2001q3,10639.5,12670.1
2001q4,10701.3,12705.3
2002q1,10834.4,12822.3


In [12]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    df = cleaned_data() 
    start = ' '
    for i in range(len(df)-2):
        if df['GDP'][i] > df['GDP'][i+1] and df['GDP'][i+1] > df['GDP'][i+2]:
            start = df.index[i]
            break
    return start

get_recession_start()

'2008q3'

In [34]:
def get_recession_end():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    df = cleaned_data()
    start = 0
    end = ' '
    for i in range(len(df)-2):
        if df['GDP'][i] > df['GDP'][i+1] and df['GDP'][i+1] > df['GDP'][i+2]:
            start = i
            break
    for i in range(start,len(df)-2):
        if df['GDP'][i+2] > df['GDP'][i+1] and df['GDP'][i+1] > df['GDP'][i]:
            end = df.index[i+2]
            break
    return end

get_recession_end()

'2009q4'

In [17]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    df = cleaned_data()
    start = 0
    end = ' '
    for i in range(len(df)-2):
        if df['Chained GDP'][i] > df['Chained GDP'][i+1] and df['Chained GDP'][i+1] > df['Chained GDP'][i+2]:
            start = i
            break
    for i in range(start,len(df)-2):
        if df['Chained GDP'][i+2] > df['Chained GDP'][i+1] and df['Chained GDP'][i+1] > df['Chained GDP'][i]:
            end = i+2
            break
    bottom = df[df['Chained GDP'] == np.min(df['Chained GDP'][start:end])]
    return bottom.index.values[0]

get_recession_bottom()

'2009q2'

In [61]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df = pd.read_csv('City_Zhvi_AllHomes.csv')
    df.replace({'State':states}, inplace = True)
    df.set_index(['State','RegionName'],inplace = True)
    df = df.loc[:, '2000-01':]
    columns_list = [str(year)+quarter for year in range(2000, 2017) for quarter in ['q1', 'q2', 'q3', 'q4']]
    del(columns_list[-1])
    
    value = 0
    for column in columns_list:
        df[column] = df.iloc[:, value:value+3].mean(axis = 1)
        value += 3
    
    df = df.loc[:, '2000q1':]
    return df

#convert_housing_data_to_quarters()

In [74]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    start = get_recession_start()
    bottom = get_recession_bottom()
    house = convert_housing_data_to_quarters()
    house = house.loc[:, start: bottom]
    house.reset_index(inplace= True)
    house['Price Ratio'] = house[start] / house[bottom]

    uni = get_list_of_university_towns()
    uni_list = uni['RegionName'].tolist()
    house['UniversityTown'] = house.RegionName.apply(lambda x: x in uni_list)

    uni_towns = house[house.UniversityTown].copy().dropna()
    non_uni_towns = house[~house.UniversityTown].copy().dropna()

    p_value = ttest_ind(uni_towns['Price Ratio'], non_uni_towns['Price Ratio'])[1]
    different = p_value < 0.01
    lower = 'university town' if uni_towns['Price Ratio'].mean() < non_uni_towns['Price Ratio'].mean() else 'non-university town'
      
    return (different, p_value, lower)

run_ttest()

(True, 0.00019452075242294589, 'university town')